<a href="https://colab.research.google.com/github/fonteleslrivka/Emergence-of-Causal-Order-from-a-Pre-Geometric-Substrate/blob/main/Full_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================
# 0. Setup — Install dependencies
# ============================================

!pip install numpy scipy matplotlib seaborn networkx --quiet


In [2]:
# ===========================================================
# 1. Imports and global configuration
# ===========================================================
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
import time, os

plt.style.use("seaborn-v0_8")
EPS = 1e-12

# Directory for figures
OUT_DIR = "figures"
os.makedirs(OUT_DIR, exist_ok=True)


In [4]:
# ===========================================================
# 2. Utilities: stable softmax, safe eigen solvers
# ===========================================================

def stable_softmax(evals, clip_min=-100, clip_max=100):
    ev = np.asarray(evals, dtype=np.float64)
    ev = np.clip(ev, clip_min, clip_max)
    m = np.max(ev)
    z = np.exp(ev - m)
    Z = np.sum(z)
    if not np.isfinite(Z) or Z == 0:
        lam = np.ones_like(ev) / ev.size
    else:
        lam = z / Z
    lam = np.clip(lam, 1e-300, 1.0)
    lam /= lam.sum()
    return lam


def safe_eigsh_dense_or_sparse(mat, k=40, which='LA'):
    n = mat.shape[0]

    if n <= 800:
        try:
            return eigh(mat, eigvals_only=True)
        except Exception:
            pass

    try:
        if not isinstance(mat, csr_matrix):
            M = csr_matrix(mat)
        else:
            M = mat

        k_try = min(max(6, min(k, n-2)), n-2)
        vals = eigsh(M, k=k_try, which=which, return_eigenvectors=False)
        return np.asarray(vals)

    except:
        return eigh(mat, eigvals_only=True)


In [5]:
# ===========================================================
# 3. Core model: random points → rho → g
# ===========================================================

def make_points(N, dim=4, seed=0):
    rng = np.random.default_rng(seed)
    X = rng.normal(0, 1, size=(N, dim))
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    X /= norms
    return X


def make_rho_g_from_points(X, beta=1.0):
    D = squareform(pdist(X, metric="euclidean")).astype(np.float32)
    rho = np.exp(-D / beta)
    rho /= np.max(rho) + EPS
    g = -np.log(rho + EPS)
    return rho, g


In [6]:
# ===========================================================
# 4. Spectral extraction helpers
# ===========================================================

def top_abs_eigvals_of_g(g, k_pos=40, k_neg=40):
    n = g.shape[0]
    k_pos = min(k_pos, max(6, n-2))
    k_neg = min(k_neg, max(6, n-2))

    try:
        vals_pos = safe_eigsh_dense_or_sparse(g, k=k_pos, which='LA')
        vals_neg = safe_eigsh_dense_or_sparse(g, k=k_neg, which='SA')
        vals = np.concatenate([vals_pos, vals_neg])
    except:
        vals = eigh(g, eigvals_only=True)

    abs_sorted = np.sort(np.abs(vals))[::-1]
    return abs_sorted


In [8]:
# ===========================================================
# 5. Projected Hessian (stability test)
# ===========================================================

def projected_hessian_fd(g0, m=6, h=1e-5, include_cross=True):
    evals, evecs = eigh(g0)
    idx = np.argsort(np.abs(evals))[::-1][:m]
    V = evecs[:, idx]

    def F_entropy(G):
        w = eigh(-G, eigvals_only=True)
        lam = stable_softmax(w)
        return -np.sum(lam * np.log(lam + 1e-300))

    F0 = F_entropy(g0)
    H = np.zeros((m, m))

    Xs = []
    for i in range(m):
        Xi = np.outer(V[:,i], V[:,i])
        Xi /= np.linalg.norm(Xi, 'fro')
        Xs.append(Xi)

    for a in range(m):
        Xa = Xs[a]
        Hp = F_entropy(g0 + h*Xa)
        Hm = F_entropy(g0 - h*Xa)
        H[a,a] = (Hp - 2*F0 + Hm)/(h*h)

    if include_cross:
        for a in range(m):
            for b in range(a+1, m):
                Xa, Xb = Xs[a], Xs[b]
                Fpp = F_entropy(g0 + h*Xa + h*Xb)
                Fpm = F_entropy(g0 + h*Xa - h*Xb)
                Fmp = F_entropy(g0 - h*Xa + h*Xb)
                Fmm = F_entropy(g0 - h*Xa - h*Xb)
                H[a,b] = (Fpp - Fpm - Fmp + Fmm)/(4*h*h)
                H[b,a] = H[a,b]

    return H, V


In [10]:
# ===========================================================
# 6. Figure Generators — Each returns data + saves PNG
# ===========================================================

def save_fig(fig, name):
    fig.savefig(os.path.join(OUT_DIR, name), dpi=150, bbox_inches="tight")
    print("Saved:", name)
    plt.close(fig)


def fig_spectrum_log(N=512, seed=0):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    eigs = top_abs_eigvals_of_g(g, k_pos=60, k_neg=60)

    fig, ax = plt.subplots(figsize=(8,4))
    ax.semilogy(eigs[:80], 'o-')
    ax.set_title(f"Spectrum |g| (log scale), N={N}")
    ax.axvline(3.5, color='red', ls='--', label="1+3 region (expected)")
    ax.legend(); ax.grid(alpha=0.3)
    save_fig(fig, "fig_spectrum_log.png")

    return eigs


In [11]:
def fig_boxplot_top12(N=512, seeds=20):
    data = []
    for s in range(seeds):
        X = make_points(N, seed=s)
        rho, g = make_rho_g_from_points(X)
        vals = top_abs_eigvals_of_g(g, 12, 12)[:12]
        data.append(vals)

    data = np.array(data)
    fig, ax = plt.subplots(figsize=(9,5))
    ax.boxplot([data[:,i] for i in range(12)])
    ax.set_yscale('log')
    ax.set_title("Seed Variability of Top 12 |λ|")
    ax.grid(alpha=0.3)
    save_fig(fig, "fig_boxplot_top12.png")

    return data


In [12]:
def fig_softmax_weights(N=512, seed=0):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    vals = top_abs_eigvals_of_g(g, 20, 20)[:20]

    lam = stable_softmax(vals)

    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(np.arange(20), lam)
    ax.set_ylim(0,1.05)
    ax.set_title("Softmax weights: eigenvalue collapse (should show λ0 ≈ 1)")
    save_fig(fig, "fig_softmax_weights.png")

    return lam


In [13]:
def fig_hessian(N=512, seed=0):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    H, V = projected_hessian_fd(g, m=6)
    w = np.linalg.eigvalsh(H)

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(w, 'o-')
    ax.set_title("Projected Hessian eigenvalues (all ≥ 0 → stable)")
    ax.grid(alpha=0.3)
    save_fig(fig, "fig_hessian_eigs.png")

    return w


In [14]:
def fig_overlap(N=512, seed=0):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)

    evals, evecs = eigh(g)
    idx = np.argsort(np.abs(evals))[::-1][:6]
    V = evecs[:, idx]
    v0 = V[:,0]

    H, _ = projected_hessian_fd(g, m=6)
    w, U = np.linalg.eig(H)
    unstable_vec = U[:, np.argmax(np.abs(w))]
    v_rec = V @ unstable_vec

    overlap = abs(np.dot(v_rec, v0)) / (np.linalg.norm(v_rec)*np.linalg.norm(v0))

    fig, ax = plt.subplots(figsize=(5,4))
    ax.bar(["overlap"], [overlap])
    ax.set_ylim(0,1)
    ax.set_title("Overlap (should be 0.0)")
    save_fig(fig, "fig_overlap_zero.png")

    return overlap


In [15]:
summary = {}

summary["spectrum"] = fig_spectrum_log()
summary["top12"] = fig_boxplot_top12()
summary["weights"] = fig_softmax_weights()
summary["H_eigs"] = fig_hessian()
summary["overlap"] = fig_overlap()

summary


Saved: fig_spectrum_log.png
Saved: fig_boxplot_top12.png
Saved: fig_softmax_weights.png
Saved: fig_hessian_eigs.png
Saved: fig_overlap_zero.png


{'spectrum': array([6.93622986e+02, 6.93622986e+02, 1.07035904e+02, ...,
        3.95476520e-02, 3.30816545e-02, 3.30816545e-02], dtype=float32),
 'top12': array([[693.623    , 693.623    , 107.035904 , 107.035904 , 105.88567  ,
         105.88567  ,  96.141624 ,  96.141624 ,  89.46035  ,  89.46035  ,
          13.276694 ,  13.276694 ],
        [693.9343   , 693.9343   , 108.05692  , 108.05692  , 104.617004 ,
         104.617004 ,  95.886375 ,  95.886375 ,  90.525345 ,  90.525345 ,
          13.915443 ,  13.915443 ],
        [693.67773  , 693.67773  , 110.67328  , 110.67328  , 104.695274 ,
         104.695274 ,  96.41737  ,  96.41737  ,  87.09888  ,  87.09888  ,
          13.473183 ,  13.473183 ],
        [693.9375   , 693.9375   , 114.19909  , 114.19909  ,  98.96061  ,
          98.96061  ,  95.81474  ,  95.81474  ,  89.92984  ,  89.92984  ,
          13.482453 ,  13.482453 ],
        [694.5238   , 694.5238   , 116.254555 , 116.254555 , 102.98753  ,
         102.98753  ,  91.1188   , 